2021-07  
本教程基于DGL官方指导'A Blitz introduction to DGL'，更换了所有过期API与中国大陆地区无法获取资源（墙）的写法，加入大量原未指明注释，在第三部分修改了并对比了各个模型

In [1]:
import numpy as np
import itertools
import scipy.sparse as sp
import dgl
import torch

[Using backend: pytorch
17:59:11] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/tianran/anaconda3/envs/Pytorch/lib/python3.7/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.9.1.so: cannot open shared object file: No such file or directory


# 同构图的处理

## 1.节点分类问题
### 加载DGL自带的引文节点数据集
引文图数据Cora由2708个出版物作为节点，共有基于案例，遗传算法，神经网络，概率方法，强化学习，规则学习，理论这七类论文，他们互相之间的引用关系为links。Feature共1433维，表示这1433个单词在这些论文中的存在情况（以0-1表示）

通常一个dataset对象包含多个图，但cora网络只有一个图，dataset[1]越界

ref:https://linqs.soe.ucsc.edu/data

In [2]:
dataset = dgl.data.citation_graph.load_cora()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


**图上的简单操作** 

In [3]:
g.ndata['feat'].shape

torch.Size([2708, 1433])

In [4]:
# Check the degrees of node in graph
g.out_degrees(240)
g.in_degrees(240)
# Induce a subgraph from node 0, node 1 and node 3 from the original graph.
sg1 = g.subgraph([0, 1, 3])
# Induce a subgraph from edge 0, edge 1 and edge 3 from the original graph.
sg2 = g.edge_subgraph([0, 1, 3])
# Add a reverse edge for each edge in the original graph
newg = dgl.add_reverse_edges(g)

**定义一个GCN(图卷积网络)**  

In [5]:
#GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to(device)
class GCN(torch.nn.Module):
    #这里由两个图卷积层组成
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = dgl.nn.pytorch.GraphConv(in_feats, h_feats)
        self.conv2 = dgl.nn.pytorch.GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = torch.nn.functional.relu(h)
        h = self.conv2(g, h)
        return h

两个困惑  
1.为何选出的training set的数目远小于validation和test  
2.将train函数中改为train_acc后可以看到acc在70个epoch到100%，再按这个训练有什么意义吗

In [6]:
print(g.ndata)
print(g.ndata['train_mask'].sum())
print(g.ndata['label'][g.ndata['train_mask']])

{'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False])}
tensor(140)
tensor([4, 4, 5, 4, 0, 6, 6, 6, 6, 6, 4, 3, 6, 6, 6, 6, 3, 6, 5, 5, 5, 5, 5, 6,
        5, 5, 2, 2, 2, 3, 1, 2, 1, 1, 4, 0, 6, 3, 6, 0, 6, 5, 5, 5, 3, 3, 5, 2,
        1, 2, 2, 2, 2, 2, 5, 5, 1, 1, 3, 0, 0, 0, 5, 3, 6, 5, 1, 0, 6, 3, 5, 1,
        1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 5, 5, 3, 6, 3, 0, 0, 4, 0, 0, 3,
    

In [15]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0
    epoch_num = 100

    features = g.ndata['feat']#节点特征
    labels = g.ndata['label']#节点类别
    train_mask = g.ndata['train_mask'].bool()#    
    #why 140？----在网络中随机选出的少数节点有助于加速计算？
    
    val_mask = g.ndata['val_mask'].bool()#验证节点500个
    test_mask = g.ndata['test_mask'].bool()#测试节点1000个
    
    #训练100个epoch
    for e in range(epoch_num):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = torch.nn.functional.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, train acc {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, train_acc, val_acc, best_val_acc, test_acc, best_test_acc))

In [16]:
g.ndata['feat'].shape[1]

1433

In [17]:
#若无支持cuda的显卡请将字段更换为'cpu'
device = torch.device('cuda:0')
# 初始化类与图结构,16为下一层-hidden_feature的长度
g = g.to(device)
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to(device)
train(g, model)

In epoch 0, loss: 1.946, train acc 0.157, val acc: 0.172 (best 0.172), test acc: 0.151 (best 0.151)
In epoch 5, loss: 1.896, train acc 0.729, val acc: 0.400 (best 0.444), test acc: 0.432 (best 0.453)
In epoch 10, loss: 1.820, train acc 0.850, val acc: 0.568 (best 0.568), test acc: 0.571 (best 0.571)
In epoch 15, loss: 1.723, train acc 0.893, val acc: 0.596 (best 0.596), test acc: 0.611 (best 0.611)
In epoch 20, loss: 1.604, train acc 0.907, val acc: 0.654 (best 0.654), test acc: 0.650 (best 0.650)
In epoch 25, loss: 1.468, train acc 0.907, val acc: 0.676 (best 0.676), test acc: 0.678 (best 0.678)
In epoch 30, loss: 1.318, train acc 0.907, val acc: 0.696 (best 0.698), test acc: 0.692 (best 0.685)
In epoch 35, loss: 1.159, train acc 0.921, val acc: 0.704 (best 0.704), test acc: 0.707 (best 0.700)
In epoch 40, loss: 1.000, train acc 0.936, val acc: 0.706 (best 0.710), test acc: 0.710 (best 0.712)
In epoch 45, loss: 0.846, train acc 0.964, val acc: 0.720 (best 0.720), test acc: 0.725 (best

**图的保存和加载**  
加载时返回的两个argment：    

graph_list (list[DGLGraph]) – The loaded graphs.

labels (dict[str, Tensor]) – The graph labels stored in file. If no label is stored, the dictionary is empty. Regardless of whether the idx_list argument is given or not, the returned dictionary always contains the labels of all the graphs.

In [8]:
# Save graphs
dgl.save_graphs('graph.dgl', g)
# Load graphs
(g,), _ = dgl.load_graphs('graph.dgl')

## 2.设计自己的图网络结构  
DGL使用了一个message passing paradigm信息传播机制来建立图神经网络，该机制首先被提出在https://arxiv.org/abs/1704.01212 ，作者认为很多GNN模式可以被一套范式描述，如下式：  
$m_{u \rightarrow v}^{(l)}=M^{(l)}\left(h_{v}^{(l-1)}, h_{u}^{(l-1)}, e_{u \rightarrow v}^{(l-1)}\right)$  
$m_{v}^{(l)}=\sum_{u \in \mathcal{N}(v)} m_{u \rightarrow v}^{(l)}$  
$h_{v}^{(l)}=U^{(l)}\left(h_{v}^{(l-1)}, m_{v}^{(l)}\right)$  

1式这里$h_{v}^{(l-1)}$表示第l-1层v号节点的的特征，u也一样；$e_{u \rightarrow v}^{(l-1)}$表示从u到v的边上信息；M为消息函数message function，它描述了一个将两个相邻节点和边的信息的Embedding传输的过程。  

而2式中的$m_{v}^{(l)}$表示传到v节点的所有相邻节点u的信息经过聚合函数(reduce function或 aggregation function即图的空域卷积)$\sum$之后的结果，这也是GraphSAGE，GAT等模型的创新点所在，就得到了下一层中的聚合过领域关系的向量$m_{v}^{(l)}$   

3式中$m_{v}^{(l)}$与原来这一节点上的特征$h_{v}^{(l-1)}$，再经过某种函数$U^{(l)}$就得到了这一节点在l层的特征 $h_{v}^{l-1}$

总结而言，除了函数M为机制给出，在新定义网络结构时我们需要指定$\sum$和U这两个函数。举例而言，在2017被提出的著名的GraphSAGE论文中，$\sum$为取平均，最大，LSTM这三类操作，而U事实上是两个特征做concat(向量拼接)操作后做一个带ReLu做激活函数的线性回归，即$h_{v}^{k} \leftarrow \operatorname{ReLU}\left(W^{k} \cdot \operatorname{CONCAT}\left(h_{v}^{k-1}, h_{\mathcal{N}(v)}^{k}\right)\right)$

ref：https://docs.dgl.ai/en/latest/tutorials/blitz/3_message_passing.html  
https://blog.csdn.net/znsoft/article/details/115996867  
https://archwalker.github.io/blog/2019/07/07/GNN-Framework-DGL-GCN.html#:~:text=DGL%20%E6%A0%B8%E5%BF%83%20%E2%80%94%20%E6%B6%88%E6%81%AF%E4%BC%A0%E9%80%92%20DGL,%E7%9A%84%E6%A0%B8%E5%BF%83%E4%B8%BA%E6%B6%88%E6%81%AF%E4%BC%A0%E9%80%92%E6%9C%BA%E5%88%B6%EF%BC%88message%20passing%EF%BC%89%EF%BC%8C%E4%B8%BB%E8%A6%81%E5%88%86%E4%B8%BA%E6%B6%88%E6%81%AF%E5%87%BD%E6%95%B0%20%EF%BC%88message%20function%EF%BC%89%E5%92%8C%E6%B1%87%E8%81%9A%E5%87%BD%E6%95%B0%EF%BC%88reduce%20function%EF%BC%89%E3%80%82

In [9]:
import dgl.function as fn
#定义一个新的网络层（自己实现GraphSAGE）
class SAGEConv(torch.nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        #这里乘2是因为要和原向量拼接
        self.linear = torch.nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        #使用local_scope() 范围时，任何对节点或边的修改在脱离这个局部范围后将不会影响图中的原始特征值 
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            # copy_u为1元操作；mean为2元操作，第一个参数只接受一个message对象
            # 详见 https://blog.csdn.net/qq_33148001/article/details/105193074
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

#一个对于信息和聚合函数的更灵活的方案是自定义函数做边和节点运算，如下        
#https://docs.dgl.ai/en/latest/api/python/udf.html#apiudf
#使用定义的图卷积层定义网络
class Model(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = torch.nn.functional.relu(h)
        h = self.conv2(g, h)
        return h

Message function fn.copy_u('h', 'm') that copies the node feature under name 'h' as messages sent to neighbors.

Reduce function fn.mean('m', '$h_N$') that averages all the received messages under name 'm' and saves the result as a new node feature '$h_N$'.

update_all tells DGL to trigger the message and reduce functions for all the nodes and edges.


In [10]:
model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.953, val acc: 0.074 (best 0.074), test acc: 0.093 (best 0.093)
In epoch 5, loss: 1.873, val acc: 0.162 (best 0.162), test acc: 0.184 (best 0.184)
In epoch 10, loss: 1.725, val acc: 0.290 (best 0.290), test acc: 0.319 (best 0.319)
In epoch 15, loss: 1.515, val acc: 0.376 (best 0.376), test acc: 0.385 (best 0.385)
In epoch 20, loss: 1.257, val acc: 0.426 (best 0.426), test acc: 0.465 (best 0.465)
In epoch 25, loss: 0.976, val acc: 0.556 (best 0.556), test acc: 0.559 (best 0.559)
In epoch 30, loss: 0.698, val acc: 0.638 (best 0.638), test acc: 0.633 (best 0.633)
In epoch 35, loss: 0.457, val acc: 0.710 (best 0.710), test acc: 0.695 (best 0.695)
In epoch 40, loss: 0.279, val acc: 0.736 (best 0.736), test acc: 0.732 (best 0.732)
In epoch 45, loss: 0.166, val acc: 0.746 (best 0.746), test acc: 0.749 (best 0.748)
In epoch 50, loss: 0.100, val acc: 0.744 (best 0.746), test acc: 0.753 (best 0.748)
In epoch 55, loss: 0.063, val acc: 0.740 (best 0.746), test acc: 0.748 (best 0

## 3.DGL进行链接预测  
**Steps：**

1.Treat the edges in the graph as positive examples.

2.Sample a number of non-existent edges (i.e. node pairs with no edges between them) as negative examples.

3.Divide the positive examples and negative examples into a training set and a test set.

4.Evaluate the model with any binary classification metric such as Area Under Curve (AUC).

In [24]:
#DGL中的图均以有向图存储，uv分别记录指向与指出边
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
#随机测试选取节点为测试集合
test_size = int(len(eids) * 0.1)
#确定test与train的positive graph的边ID(由存在的边构成的图)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

#计算图的邻接矩阵
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
#这是图上所有不存在的边
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
#(neg_u[x], neg_v[x])对应矩阵里一个点
neg_u, neg_v = np.where(adj_neg != 0)
#随机选择一个edge的ID
neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
#确定test与train的negtive graph的边ID(由不存在的边构成的图)
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[train_size:]], neg_v[neg_eids[train_size:]]

#构建一个训练用图，将edge去掉test集合中的edge
train_g = dgl.remove_edges(g, eids[:test_size])

在DGL中，train与test都将由neg与pos组成，他们都由原始图节点组成，这也就是说，DGL中区分这两个图的关键在于边，这训练，测试集各两组的边公用各自的节点集

In [14]:
#这里不自己构建网络层，直接使用内嵌的API
# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = dgl.nn.SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = dgl.nn.SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = torch.nn.functional.relu(h)
        h = self.conv2(g, h)
        return h
#按照Index正式划分
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

在DGL中，如果想对边信息做进一步处理整合，API提供了两种方案  
1.直接使用内建函数：dgl.function.fn.xxxxxx  >   apply_edges    
2.在复杂操作时自定义类: 注意自定义的apply_edges函数的返回参数格式

下面的代码将节点信息与边相关联，详情见训练部分注释

In [45]:
import dgl.function as fn
#使用DGL自建函数
class DotPredictor(torch.nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

#自定义
class MLPPredictor(torch.nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = torch.nn.Linear(h_feats * 2, h_feats)
        self.W2 = torch.nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(torch.nn.functional.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

自定义损失与计算AUC

In [32]:
train_g.ndata['feat'].shape[1]

1433

In [47]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 20)
#以两种方式给出预测结果，Dot的效果好于多层感知机，这里应该是训练量太小
#MLP的中间向量维度16如果要更改，需要在上一行model定义中也将SAGE的输出向量维度更改
#在Dot的方案中直接更改model定义即可，参数在一定范围内变化时对精度没有大范围改变
pred = DotPredictor()
#pred = MLPPredictor(16)
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return torch.nn.functional.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    # 模型返回一个经过了2层SAGE层的特征向量h,大小为[2708, 16]，即2708个节点，每个节点经过网络后由16维向量表示
    h = model(train_g, train_g.ndata['feat'])
    #用两个节点所表示的向量，两个1×16的矩阵做点积，得到一个1×1的矩阵，一个数，作为这个边出现的概率score
    #分别计算图上原来存在与不存在的边，以维持正负样本均衡
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    #计算交叉熵损失
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.6912696957588196
In epoch 5, loss: 0.40669772028923035
In epoch 10, loss: 0.3272048234939575
In epoch 15, loss: 0.30698299407958984
In epoch 20, loss: 0.3082151412963867
In epoch 25, loss: 0.2948625683784485
In epoch 30, loss: 0.276272177696228
In epoch 35, loss: 0.26275402307510376
In epoch 40, loss: 0.24984697997570038
In epoch 45, loss: 0.2334848940372467
In epoch 50, loss: 0.21313920617103577
In epoch 55, loss: 0.19108596444129944
In epoch 60, loss: 0.172286257147789
In epoch 65, loss: 0.15656094253063202
In epoch 70, loss: 0.14071063697338104
In epoch 75, loss: 0.12661157548427582
In epoch 80, loss: 0.11184187978506088
In epoch 85, loss: 0.09554004669189453
In epoch 90, loss: 0.07968775928020477
In epoch 95, loss: 0.0655575543642044
AUC 0.8190939107387525
